# Dataset Analisys

> to start, let's import all libraries we'll use, as well as our dataset


### Import libraries

In [1]:
import pandas as pd
import json
from pathlib import Path
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
import re
import matplotlib.pyplot as plt

### Import dataset from JSON

In [2]:
JSON_PATH = Path("data/raw.json")

# 1. Open JSON
with open(JSON_PATH, "r", encoding="utf-8") as f:
    data = json.load(f)

# 2. Proceed only if it's a list of records
if isinstance(data, dict):
    data = list(data.values())
elif not isinstance(data, list):
    raise ValueError("The JSON file seems to be empty or not a list of records.")

# 3. Convert to DataFrame, normalizing
df = pd.json_normalize(data)

# 4. Remove empty columns, or columns with all NaN values
df.columns = [col.strip() for col in df.columns]
df = df.loc[:, ~df.columns.str.fullmatch(r"Unnamed:.*|^$")]
df = df.dropna(axis=1, how="all")

### General info about the dataset

In [ ]:
print(f"Il dataframe ha {df.shape[1]} colonne.")
print(f"Il dataframe ha {df.shape[0]} righe.")

# Rename some columns
df = df.rename(columns={
    "Power HP": "Power_HP",
    "Price as new": "Price"
})

print(f"La moto più vecchia è del {df['Year'].min()}")
print(f"La moto più recente è del {df['Year'].max()}.")


display(df.columns)
display(df["Power"])
#CONVERTIRE POWER IN UN NUMERO DI CAVALLI
#CONVERTIRE IL PREZZO SE NECESSARIO

Il dataframe ha 82 colonne.
Il dataframe ha 38624 righe.
La moto più vecchia è del 1894
La moto più recente è del 2021.


Index(['Model', 'Year', 'Category', 'Rating', 'Displacement', 'Engine type',
       'Torque', 'Bore x stroke', 'Fuel system', 'Fuel control',
       'Cooling system', 'Gearbox', 'Transmission type,final drive',
       'Frame type', 'Rake (fork angle)', 'Trail', 'Front suspension',
       'Rear suspension', 'Rear wheel travel', 'Front tyre', 'Rear tyre',
       'Front brakes', 'Diameter', 'Rear brakes', 'Seat height',
       'Ground clearance', 'Wheelbase', 'Fuel capacity', 'Color options',
       'Starter', 'Comments', 'Insurance costs', 'Finance options',
       'Parts finder', 'Ask questions', 'Related bikes', 'Price',
       'Engine details', 'Power', 'Compression', 'Valves per cylinder',
       'Ignition', 'Lubrication system', 'Clutch', 'Driveline',
       'Fuel consumption', 'Greenhouse gases', 'Emission details',
       'Exhaust system', 'Front wheel travel', 'Wheels', 'Dry weight',
       'Weight incl. oil, gas, etc', 'Power/weight ratio', 'Overall height',
       'Overall leng

0                                    NaN
1         95.2 HP (69.5  kW)) @ 8750 RPM
2                    28.2 HP (20.6  kW))
3        15.0 HP (10.9  kW)) @ 10500 RPM
4                                    NaN
                      ...               
38619        2.5 HP (1.8  kW)) @ 240 RPM
38620                  0.5 HP (0.4  kW))
38621        2.5 HP (1.8  kW)) @ 240 RPM
38622        1.2 HP (0.9  kW)) @ 180 RPM
38623        2.5 HP (1.8  kW)) @ 240 RPM
Name: Power, Length: 38624, dtype: object

### Define useful functions

In [4]:
# Convert to numeric
def to_float(txt, unit=None):
    if pd.isna(txt): return None
    m = re.search(r"[-+]?\d*\.?\d+", str(txt))
    if not m: return None
    num = float(m.group())
    if unit=="in_to_mm":   num *= 25.4
    if unit=="lbft_to_Nm": num *= 1.35582
    return num

# We use this function to make the fonts bigger, as well as changing the background color
def set_theme(fig, font_size=20, bg_color="#141415", font_color="white", grid_color="#333333"):
    fig.update_layout(
        paper_bgcolor=bg_color,
        plot_bgcolor=bg_color,
        font=dict(size=font_size, color=font_color),
        xaxis=dict(gridcolor=grid_color, zerolinecolor=grid_color),
        yaxis=dict(gridcolor=grid_color, zerolinecolor=grid_color),
        legend=dict(bgcolor="rgba(0,0,0,0)")  # Transparent legend background
    )
    return fig


### Cleaning of dataset and inizializations

In [ ]:

# Convert certain columns to numeric
df["Displacement_cc"]   = df["Displacement"].apply(to_float)
df["Torque_Nm"]         = df["Torque"].apply(lambda x: to_float(x, unit="lbft_to_Nm") or to_float(x))
df["Seat_height_mm"]    = df["Seat height"].apply(to_float)
df["Fuel_capacity_l"]   = df["Fuel capacity"].apply(to_float)

# palette & template
TEMPLATE = "presentation"
pio.templates.default = TEMPLATE



## Charts
 > Let's finnally see some charts

### Models by category
 > Let's see the most common type of motorcycle, leaving out those that appear less in the dataset


In [11]:
# Group by most used, leave the others in "Other"
df_grouped = df.groupby("Category").size().reset_index(name="count")

df_top_types = df_grouped[df_grouped["count"] >= 1500].sort_values(by="count", ascending=False)
other_types = df_grouped[df_grouped["count"] < 1500].sum()["count"]

# Create a new DataFrame with the top types and the "Other" category
df_grouped = pd.concat([df_top_types, pd.DataFrame([{'Category': 'Other', 'count': other_types}])], ignore_index=True)

# Concat the top types and the "Other" category
df_pie = pd.concat(
    [df_top_types, pd.DataFrame([{"Category": "Other", "count": other_types}])],
    ignore_index=True
)

# Sort the DataFrame by count in descending order
ordered_categories = list(df_top_types["Category"]) + ["Other"]
df_pie["Category"] = pd.Categorical(
    df_pie["Category"], categories=ordered_categories, ordered=True
)
gray_seq = ["gray"] + ["gray"] + ["gray"] + ["gray"] + ["white"]
color_seq = px.colors.qualitative.Plotly[:5] + gray_seq  # bianco per Other


# Create the pie chart
fig1 = px.pie(
    df_pie,
    names="Category",
    values="count",
    title="<b>Models by Type of bike</b>",
    hole=0.35,
    color_discrete_sequence=color_seq,
    category_orders={"Category": ordered_categories},
)

# Rotate the pie chart, make it counterclockwise, don't atutomatically sort
fig1.update_traces(
    direction="counterclockwise",
    sort=False,
    rotation=-61
)


set_theme(fig1).show()

### Counts of bikes per cc
 > Let's figure out how many bikes were produced per cc category

In [15]:
bins   = [0, 125, 400, 700, 1000, 2000, np.inf]
labels = ["0–125 (A1)", "125–400 (A-lim)", "400–700 (A)", 
          "700–1000 (A)", "1000-2000 (A)", "2000+ (A)"]

# A new column with the categories in which every model falls
df["Disp_cat"] = pd.cut(df["Displacement_cc"], bins=bins, labels=labels, right=True)

# For each category, we calculate the count and the mode            
count_df  = (df["Disp_cat"]
             .value_counts()
             .reindex(labels)
             .rename("Count")
             .reset_index()
             .rename(columns={"Disp_cat": "Displacement Range (cc)"}))

# Calulates the mode for each category
mode_df = (
    df.groupby("Disp_cat", observed=False)["Displacement_cc"]
    .agg(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    .reindex(labels)
    .reset_index()
    .rename(columns={
        "Disp_cat": "Displacement Range (cc)",
        "Displacement_cc": "mode_cc"
    })
)

# Merge the two DataFrames
disp_summary = count_df.merge(mode_df, on="Displacement Range (cc)")
# Display the mode in a more readable format
disp_summary["mode_text"] = "Mode: " + disp_summary["mode_cc"].round(0).astype(int).astype(str)


fig2 = px.bar(
    disp_summary,
    x="Displacement Range (cc)",
    y="Count",
    #This will show the mode on the bar
    text="mode_text",

    title="<b>Count of Models by CC</b>",
    color_discrete_sequence=["#987434"],
    height=650
)

fig2.update_xaxes(categoryorder="array", categoryarray=labels)

fig2.update_traces(
    texttemplate="%{text}", 
    textposition="inside",
    insidetextanchor="middle", 
    textfont=dict(color="white", size=20)
)



set_theme(fig2).show()


### Price related to a lot of stuff

In [17]:
# anno come x, prezzo come grandezza, y come cc, colore power

plt.figure(figsize=(20, 15))
plt.scatter(
    df["Year"],
    df["Displacement_cc"],
    c=df["Power"],
    s=df["Price_EUR"],
    alpha=0.3, 
    cmap='viridis', 
    edgecolors='k'
    
    )
plt.xlabel('Years')
plt.ylabel('CC')
plt.title('Blood Pressure vs. Heart Rate')
plt.colorbar()

KeyError: 'Price_EUR'

<Figure size 2000x1500 with 0 Axes>